**CCF​353 - ORGANIZAÇÃO DE COMPUTADORES II**

**Bacharelado em Ciência da Computação**

**Universidade Federal de Viçosa - Campus Florestal**

**Prof. José Augusto​ M. Nacif - jnacif@ufv.br**

# Atividades Práticas em GPU usando CUDA

Este conteúdo foi baseado no [*colab*](https://colab.research.google.com/drive/1caBu4aCskJMyojYbU55a0aexxGZY0L_M) do Professor Ricardo Ferreira, da UFV - Campus Viçosa, e adaptado com exemplos do livro Professional CUDA C Programming. Ao longo dos exemplos, serão deixadas referências aos capítulos do livro que abordam cada assunto, para uma consulta mais detalhada.


## Introdução // Usando o Google Colab

Esta plataforma é um serviço de nuvem gratuito hospedado pelo Google baseado no Jupyter Notebook, possuindo suporte ao Python 2.7 e 3.6. Além disso, você tem acesso grátis a uma GPU, que será nosso alvo de estudo.

Depois de criar uma cópia deste colab para seu drive, você poderá editar as células de texto, editar as células de código e executá-las. Há suporte para código em Python, comandos bash, e qualquer outra linguagem, desde haja os plugins adequados, usando os comandos adequados.

Para criar uma nova célula de texto ou código, basta aproximar o mouse do fim ou início de uma célula já existente, e você poderá escolher que tipo de célula criar.

Para executar as células de código, basta clicar no botão 'play' no canto superior esquerdo dessas células. As células de texto têm suporte à Markdown e HTML.

## Introdução // Preparando o ambiente
Primeiramente, certifique-se que este colab está configurado com uma GPU: acesse *Runtime > Change runtime type > Hardware accelerator* e selecione GPU.

Usaremos um plugin desenvolvido por Andrei Nechaev para executar códigos do CUDA. Para isso, você deve executar os comandos a seguir, para baixar o plugin pelo git e depois ativá-lo com o comando `load_ext`.

Observação: você deve executar estes comandos sempre que começar a usar este colab.

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-uq53d1rl
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-uq53d1rl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=35816bef59572300769d0d7496930fd47b09e57cc54101f6baaa2ad257428cd3
  Stored in directory: /tmp/pip-ephem-wheel-cache-g9ttoh6n/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Pronto! Agora você pode adicionar células de código CUDA, basta digitar `%%cu` na primeira linha do código.

## Introdução // Programação com CUDA

_Referência: Capítulo 1 - Heterogeneous Parallel Computing with CUDA_

É importante entender alguns termos e conceitos da programação com CUDA para seguir adequadamente os exemplos e tarefas.

A ideia do CUDA é permitir ao programador trabalhar com uma arquitetura computacional heterogênea, que contém partes de código a ser executado na CPU e partes de código a ser executado na GPU.

As GPU's não são substitutas das CPU's, muito pelo contrário, o melhor é tirar proveito das especialidades de cada uma, usando uma GPU em conjunto com uma CPU. Por esse motivo, a grosso modo, damos o nome de _host_ (hospedeiro) à CPU, e _device_ (dispositivo) à GPU, pois ela será um _dispositivo_ acelerando seu _hospedeiro_.

![Diagrama de Arquitetura Heterogênea](https://nanxiao.me/wp-content/uploads/2016/12/Capture.jpg)

Um programa do CUDA então é divido em duas partes principais, o _host code_ - executado na CPU - e o _device code_ - executado na GPU. O _device code_ é escrito com palavras reservadas específicas para demarcar funções com paralelismo de dados, e a estas é dado o nome de _kernels_.

Uma métrica importante é a _capability_ de uma GPU, que mede seu poder computacional. Para simplificar, a NVIDIA criou uma numeração de _capability_ para as versões seus dispositivos: 1.0, 2.0, 3.0, 3.5, etc. A Tesla K40, por exemplo, possui _capability_ 3.5, outras GPU's da arquitetur Turing tem _capability_ 7.5. Todos exemplos mostrados serão compatíveis com _capability_ 2.0 ou superior.

Outro conceito a se entender é o de blocos e threads. Cada bloco contém várias threads e cada thread executa um mesmo trecho de código. No exemplo a seguir será exibido o funcionamento das threads.

## Exemplo 1 // Hello World!

_Referência: Capítulo 1 - Heterogeneous Parallel Computing with CUDA_

Vamos começar com um exemplo simples do livro base. Segue abaixo o código do "Hello World" modificado para imprimir o número do thread e o número do bloco de onde está sendo executado.

Observação: a macro `CHECK` serve para verificar erros nas chamadas de funções do CUDA. Essa macro será utilizada apenas aqui, mas você pode copiá-la e usá-la para verificar erros em qualquer chamada à API do CUDA. Neste exemplo, a chamada `cudaDeviceReset()` é verificada.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define CHECK(call) { \
  const cudaError_t error = call; \
  if (error != cudaSuccess) { \
    printf("Error: %s:%d, ", __FILE__, __LINE__); \
    printf("code:%d, reason: %s\n", error, cudaGetErrorString(error)); \
    exit(1); \
  } \
}

__global__ void helloFromGPU() {
  printf("Hello World from GPU! %d %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<4, 6>>>();
  CHECK(cudaDeviceReset());
  return 0;
}

Hello World from CPU!
Hello World from GPU! 3 0
Hello World from GPU! 3 1
Hello World from GPU! 3 2
Hello World from GPU! 3 3
Hello World from GPU! 3 4
Hello World from GPU! 3 5
Hello World from GPU! 0 0
Hello World from GPU! 0 1
Hello World from GPU! 0 2
Hello World from GPU! 0 3
Hello World from GPU! 0 4
Hello World from GPU! 0 5
Hello World from GPU! 1 0
Hello World from GPU! 1 1
Hello World from GPU! 1 2
Hello World from GPU! 1 3
Hello World from GPU! 1 4
Hello World from GPU! 1 5
Hello World from GPU! 2 0
Hello World from GPU! 2 1
Hello World from GPU! 2 2
Hello World from GPU! 2 3
Hello World from GPU! 2 4
Hello World from GPU! 2 5



Podemos ver que este é um código em C com uma diferença nas chamadas de funções executadas pela GPU.

No exemplo acima, a função é chamada com a sintaxe `<<<4, 6>>>`, ou seja, com 4 blocos, sendo que cada bloco tem 6 threads, o que fará com que sejam executadas 4 * 6 = 24 cópias da função. Cada cópia terá um ID único, composto pelo número do bloco (0, 1, 2 ou 3) e número do thread (0, 1, 2, 3, 4 ou 5).

O especificador `__global__` na definição da função diz ao compilador que a função seráá chamada pela CPU e executada na GPU.

A execução das várias instâncias da função `helloFromGPU` são em paralelo com a CPU. No exemplo, a CPU irá aguardar pois chama a função `cudaDeviceReset`. Porém qualquer código que for inserido entre a chamada `cudaDeviceReset` e `helloFromGPU` será executado em paralelo.

A função `cudaDeviceReset()` vai explicitamente destruir e limpar todos os recursos associados ao dispositivo usado no processo corrente.

Se, ao invés de executar diretamente o código, você quiser salvá-lo, use o comando `%%writefile caminho/para/o/arquivo`. Por padrão, o plugin cria a pasta `/content/src/` para colocar os códigos. Você pode então substituir o comando `%%cu` por `%%writefile /content/src/hello.cu`, por exemplo.

Isso é útil se você quiser especificar opções de compilação ou extrair informações sobre a execução do seu programa - o que fará parte dos exercícios. Por exemplo, após salvar o exemplo anterior com o comando `writefile` mostrado, você poderia compilar da seguinte forma:

In [ ]:
%%writefile /content/src/hello.cu
#include <stdio.h>
#include <stdlib.h>

#define CHECK(call) { \
  const cudaError_t error = call; \
  if (error != cudaSuccess) { \
    printf("Error: %s:%d, ", __FILE__, __LINE__); \
    printf("code:%d, reason: %s\n", error, cudaGetErrorString(error)); \
    exit(1); \
  } \
}

__global__ void helloFromGPU() {
  printf("Hello World from GPU! %d %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<4, 6>>>();
  CHECK(cudaDeviceReset());
  return 0;
}

Overwriting /content/src/hello.cu


In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o hello hello.cu
!./hello

/content/src
Hello World from CPU!
Hello World from GPU! 3 0
Hello World from GPU! 3 1
Hello World from GPU! 3 2
Hello World from GPU! 3 3
Hello World from GPU! 3 4
Hello World from GPU! 3 5
Hello World from GPU! 0 0
Hello World from GPU! 0 1
Hello World from GPU! 0 2
Hello World from GPU! 0 3
Hello World from GPU! 0 4
Hello World from GPU! 0 5
Hello World from GPU! 1 0
Hello World from GPU! 1 1
Hello World from GPU! 1 2
Hello World from GPU! 1 3
Hello World from GPU! 1 4
Hello World from GPU! 1 5
Hello World from GPU! 2 0
Hello World from GPU! 2 1
Hello World from GPU! 2 2
Hello World from GPU! 2 3
Hello World from GPU! 2 4
Hello World from GPU! 2 5


Com a opção `-arch`, você pode selecionar uma arquitetura. Usando uma arquitetura com _capability_ maior ou igual à 2.0, você pode usar o `printf` dentro do código da GPU - que chamammos de kernel. No comando de exemplo, foi usada a capability 3.5. Porém, como a GPU pode executar muitos threads - na ordem dos bilhões -, para exemplos maiores, é melhor não usá-lo, exceto se for condicionalmente para verificar apenas alguns threads. Por exemplo:

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void helloFromGPU() {
  if (blockIdx.x == 2048 && threadIdx.x == 512)
    printf("Hello World from GPU! Block X: %d, Thread X: %d\n",
           blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<64 * 1024 * 1024, 1024>>>();
  cudaDeviceReset();
  return 0;
}

Hello World from CPU!
Hello World from GPU! Block X: 2048, Thread X: 512



O trecho acima irá executar com 64 M (2^26) blocos com 1024 threads em cada bloco, ou seja, serão 64 G (2^36) execuções da função `helloFromGPU`. Ao longo de todo o texto, os prefixos K, M, G, etc. sempre vão se referir aos seus valores em potência de 2. Ou seja, K = 2^10, M = 2^20, G = 2^30, etc.

## Tarefa 1 // Hello World!

1. Modifique o exemplo para executar 1 bloco com 1024 threads. Imprima e verifique a ordem de execução dos threads. É sequencial? É determinística? É agrupada?


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define CHECK(call) { \
  const cudaError_t error = call; \
  if (error != cudaSuccess) { \
    printf("Error: %s:%d, ", __FILE__, __LINE__); \
    printf("code:%d, reason: %s\n", error, cudaGetErrorString(error)); \
    exit(1); \
  } \
}

__global__ void helloFromGPU() {
  printf("Hello World from GPU! %d %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<1, 1024>>>();
  CHECK(cudaDeviceReset());
  return 0;
}

Hello World from CPU!
Hello World from GPU! 0 192
Hello World from GPU! 0 193
Hello World from GPU! 0 194
Hello World from GPU! 0 195
Hello World from GPU! 0 196
Hello World from GPU! 0 197
Hello World from GPU! 0 198
Hello World from GPU! 0 199
Hello World from GPU! 0 200
Hello World from GPU! 0 201
Hello World from GPU! 0 202
Hello World from GPU! 0 203
Hello World from GPU! 0 204
Hello World from GPU! 0 205
Hello World from GPU! 0 206
Hello World from GPU! 0 207
Hello World from GPU! 0 208
Hello World from GPU! 0 209
Hello World from GPU! 0 210
Hello World from GPU! 0 211
Hello World from GPU! 0 212
Hello World from GPU! 0 213
Hello World from GPU! 0 214
Hello World from GPU! 0 215
Hello World from GPU! 0 216
Hello World from GPU! 0 217
Hello World from GPU! 0 218
Hello World from GPU! 0 219
Hello World from GPU! 0 220
Hello World from GPU! 0 221
Hello World from GPU! 0 222
Hello World from GPU! 0 223
Hello World from GPU! 0 0
Hello World from GPU! 0 1
Hello World from GPU! 0 2
Hell

Resposta. Apresenta grupos de execução sequencial de 32 threads. É possível determinar isso, porém não é possível dizer nada sobre quais grupos executarão primeiro. Provavelmente cada bloco da GPU tem apenas 32 threads, então o compilador alocará vários para rodar todos os 1024 threads solicitados.

2. Agora repita o exercício anterior com 1024 blocos com 1 thread cada. Como é a ordem de execução dos threads? Sequencial? Determinística? Agrupada?

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define CHECK(call) { \
  const cudaError_t error = call; \
  if (error != cudaSuccess) { \
    printf("Error: %s:%d, ", __FILE__, __LINE__); \
    printf("code:%d, reason: %s\n", error, cudaGetErrorString(error)); \
    exit(1); \
  } \
}

__global__ void helloFromGPU() {
  printf("Hello World from GPU! %d %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<1024, 1>>>();
  CHECK(cudaDeviceReset());
  return 0;
}

Hello World from CPU!
Hello World from GPU! 127 0
Hello World from GPU! 4 0
Hello World from GPU! 117 0
Hello World from GPU! 128 0
Hello World from GPU! 122 0
Hello World from GPU! 9 0
Hello World from GPU! 125 0
Hello World from GPU! 129 0
Hello World from GPU! 6 0
Hello World from GPU! 140 0
Hello World from GPU! 121 0
Hello World from GPU! 143 0
Hello World from GPU! 124 0
Hello World from GPU! 193 0
Hello World from GPU! 18 0
Hello World from GPU! 196 0
Hello World from GPU! 8 0
Hello World from GPU! 25 0
Hello World from GPU! 123 0
Hello World from GPU! 119 0
Hello World from GPU! 126 0
Hello World from GPU! 120 0
Hello World from GPU! 150 0
Hello World from GPU! 154 0
Hello World from GPU! 166 0
Hello World from GPU! 69 0
Hello World from GPU! 195 0
Hello World from GPU! 12 0
Hello World from GPU! 201 0
Hello World from GPU! 200 0
Hello World from GPU! 157 0
Hello World from GPU! 21 0
Hello World from GPU! 202 0
Hello World from GPU! 37 0
Hello World from GPU! 2 0
Hello World fr

Resposta. Executa de forma totalmente imprevisível. Não é sequencial. Não é determinística. Não é agrupada.

3. Compile com capability 3.5 e gere 2 trilhões de threads - que é o máximo que a GPU poderá executar -, em qualquer combinação de blocos x threads. Use um comando condicional para imprimir apenas alguns threads.

In [ ]:
%%writefile hello13.cu
#include <stdio.h>
#include <stdlib.h>


__global__ void helloFromGPU() {
  if (blockIdx.x == (1000000000-1) && threadIdx.x == (1000-1))
    printf("Hello World from GPU! Block X: %d, Thread X: %d\n",
           blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<1000000000, 1000>>>();
  cudaDeviceReset();
  return 0;
}

Overwriting hello13.cu


In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o hello13 hello13.cu
!./hello13

/content/src
Hello World from CPU!
Hello World from GPU! Block X: 999999999, Thread X: 999


4. Execute alguma operação à sua escolha na GPU, mostrando resultados, e relate-os.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define CHECK(call) { \
  const cudaError_t error = call; \
  if (error != cudaSuccess) { \
    printf("Error: %s:%d, ", __FILE__, __LINE__); \
    printf("code:%d, reason: %s\n", error, cudaGetErrorString(error)); \
    exit(1); \
  } \
}

__global__ void helloFromGPU() {
  printf("Olá mundo do heavy metal, aqui e a GPU! %d %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv) {
  printf("Hello World from CPU!\n");
  helloFromGPU<<<1, 1>>>();
  CHECK(cudaDeviceReset());
  return 0;
}

Hello World from CPU!
Olá mundo do heavy metal, aqui e a GPU! 0 0



## Exemplo 2 // Soma de Vetores

_Referência: Capítulo 2 - CUDA Programming Model_

O próximo exemplo já usa o modelo cunhado pela NVIDIA, o _Single Instruction Multiple Thread_ (SIMT), para realizar um cálculo simples em um grande conjunto de dados. Faremos a soma de dois vetores e armazená-la em outro.

A forma convencional de se fazer isso na CPU seria
``` cpp
void sumArraysOnHost(float *A, float *B, float *C, int N) {
  for (int i = 0; i < N; i++)
    C[i] = A[i] + B[i];
}
```
Porém, com a GPU, vamos desfazer o _loop_ e realizar a soma usando o ID de bloco e thread de cada thread para selecionar qual posição do vetor será somada.

Para entender como fazer esse mapeamento entre threads e posições do vetor, é necessário entender a hierarquia das threads. A figura a seguir, extraída do livro, resume bem essa ideia.

![Hierarquia de threads](https://docs.google.com/uc?export=download&id=13qryIegnVrDgmaKA7eEcV3P03ddRe3MS)

Os blocos são organizados numa grade, e cada bloco contém várias threads. Grades e blocos podem ter até três dimensões, mas geralmente as grades são 2D e os blocos são 3D. Na figura, temos uma grade 2D e um bloco também 2D.

Neste exemplo, no entanto, vamos usar uma configuração 1D, pois nossos vetores são unidimensionais. A figura abaixo ilustra essa configuração:

![Configuraçãao 1D de threads](https://docs.google.com/uc?export=download&id=1BA6xd8T--b0Y4YaUEa-O5HPiteCceohk)

Esses ID's de bloco e de thread são acessados dentro de um _kernel_ com as variáveis vistas no exemplo 1, `blockIdx` e `threadIdx`, respectivamente. Como estamos interessados apenas em uma dimensão, selecionamos apenas a dimensão `x` - as outras dimensões são `y` e `z`. Além dessas, também temos as variáveis `gridDim` e `blockDim`, que informam o tamanho da grade e do bloco, respectivamente, e também tem os campos `x`, `y` e `z`. O código da soma ficaria então:

``` cpp
__global__ void sumArraysOnGPU(float *A, float *B, float *C) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  C[i] = A[i] + B[i];
}
```
Repare também que no código não há referência ao tamanho dos vetores, já que o valor N é implicitamente definido pela quantidade de blocos e threads usadas para a soma. Por exemplo, se o vetor tiver tamanho 32, você poderia invocar o _kernel_ com qualquer uma das seguintes linhas, dentre outras:
``` cpp
sumArraysOnGPU<<<1,32>>>(A, B, C);
sumArraysOnGPU<<<4,8>>>(A, B, C); // corresponde à figura
sumArraysOnGPU<<<16,2>>>(A, B, C);
```
Para colocar esse _kernel_ em utilização, é necessário usar outras funções da API do CUDA. Serão mostrados comentários explicando em alto nível o que cada uma faz. Note que para usá-las é necessário incluir a biblioteca `cuda_runtime.h`.

In [ ]:
%%cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Soma os vetores na CPU
void sumArraysOnHost(float *A, float *B, float *C, const int N) {
  for (int i = 0; i < N; i++)
    C[i] = A[i] + B[i];
}

// Soma os vetores na GPU
__global__ void sumArraysOnGPU(float *A, float *B, float *C) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  C[i] = A[i] + B[i];
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = 2048;
  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Soma os vetores na CPU para conferir os resultados
  sumArraysOnHost(h_A, h_B, hostRef, nElem);
  // Compara os resultados
  checkResult(hostRef, gpuRef, nElem);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

/tmp/tmpg1_ooyzm/3057d6bc-4fd9-49e5-a154-1507e8e43700.out Starting...
Vector size 2048
Execution configuration <<<2, 1024>>>
Arrays match.




Observação: ao modificar este exemplo, tome cuidado para não usar uma configuração de tamanho de bloco que exceda 1024 threads por bloco. Aqui no Google Colab, os kernels não são chamados quando isso acontece. Para usar mais threads, use mais blocos por grade, fazendo os cálculos adequados.

## Tarefa 2 // Soma de Vetores

Para realizar esta tarefa, você vai usar a ferramenta `nvprof`. Para usá-la, você precisa compilar seu programa com o `nvcc` - como foi descrito anteriormente - e passar o nome do executável gerado para o `nvprof`. Por exemplo, se você salvar o código fonte do exemplo de soma de vetores como `/content/src/sum.cu`, fica assim o uso do `nvprof`:

In [ ]:
%%writefile /content/src/sum.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Soma os vetores na CPU
void sumArraysOnHost(float *A, float *B, float *C, const int N) {
  for (int i = 0; i < N; i++)
    C[i] = A[i] + B[i];
}

// Soma os vetores na GPU
__global__ void sumArraysOnGPU(float *A, float *B, float *C) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  C[i] = A[i] + B[i];
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Soma os vetores na CPU para conferir os resultados
  sumArraysOnHost(h_A, h_B, hostRef, nElem);
  // Compara os resultados
  checkResult(hostRef, gpuRef, nElem);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sum.cu


O `nvprof` vai exibir um relatório completo dos tempos de execução de cada chamada à API do CUDA e de cada kernel. Nessa tarefa você vai explorar como avaliar os tempos com o `nvprof`.

1. Compile e execute o exemplo de soma de vetores como está, e veja o desempenho com o `nvprof`. Quanto tempo foi gasto em transferências _host to device_ e _device to host_? Qual foi a taxa de transferência, em elementos do vetor por unidade de tempo? Qual foi o tempo de execução do kernel?

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sum sum.cu
!nvprof ./sum 2048

/content/src
./sum Starting...
Vector size 2048
==13320== NVPROF is profiling process 13320, command: ./sum 2048
Execution configuration <<<2, 1024>>>
Arrays match.

==13320== Profiling application: ./sum 2048
==13320== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   47.47%  6.9120us         2  3.4560us  3.3920us  3.5200us  [CUDA memcpy HtoD]
                   26.59%  3.8720us         1  3.8720us  3.8720us  3.8720us  [CUDA memcpy DtoH]
                   25.93%  3.7760us         1  3.7760us  3.7760us  3.7760us  sumArraysOnGPU(float*, float*, float*)
      API calls:   98.99%  130.23ms         3  43.412ms  4.0660us  130.22ms  cudaMalloc
                    0.45%  594.30us         1  594.30us  594.30us  594.30us  cuDeviceTotalMem
                    0.26%  341.29us        96  3.5550us     133ns  151.10us  cuDeviceGetAttribute
                    0.17%  225.93us         3  75.309us  5.7220us  132.43us  cudaFree
      

*   Host para device: 7.2960us
*   Device para host: 3.6480us
*   Tempo de transferência total: 10.944us
*   Elementos do vetor: 2048
*   Taxa de transferêcia: 188 elemetos/us
*   Tempo de execução do kernel: 14.449us


  

2. Modifique o tamanho dos vetores para 100 M, 200 M e 300 M elementos. Repita para cada tamanho as análises de tempos e taxas do exercício anterior.

In [ ]:
%cd /content/src
!nvprof --print-gpu-summary ./sum 128000000

/content/src
./sum Starting...
Vector size 128000000
==13343== NVPROF is profiling process 13343, command: ./sum 128000000
Execution configuration <<<125000, 1024>>>
Arrays match.

==13343== Profiling application: ./sum 128000000
==13343== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.49%  305.59ms         1  305.59ms  305.59ms  305.59ms  [CUDA memcpy DtoH]
                   31.82%  148.50ms         2  74.248ms  71.194ms  77.302ms  [CUDA memcpy HtoD]
                    2.69%  12.533ms         1  12.533ms  12.533ms  12.533ms  sumArraysOnGPU(float*, float*, float*)


*   Host para device: 250.54ms
*   Device para host: 109.06ms
*   Tempo de transferência total: 359,6ms
*   Elementos do vetor: 100000000
*   Taxa de transferêcia: 278087 elementos/ms
*   Tempo de execução do kernel: 369.4ms


  

In [ ]:
%cd /content/src
!nvprof --print-gpu-summary ./sum 256000000

/content/src
./sum Starting...
Vector size 256000000
==13366== NVPROF is profiling process 13366, command: ./sum 256000000
Execution configuration <<<250000, 1024>>>
Arrays match.

==13366== Profiling application: ./sum 256000000
==13366== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   66.61%  641.91ms         1  641.91ms  641.91ms  641.91ms  [CUDA memcpy DtoH]
                   30.78%  296.59ms         2  148.30ms  145.15ms  151.44ms  [CUDA memcpy HtoD]
                    2.61%  25.166ms         1  25.166ms  25.166ms  25.166ms  sumArraysOnGPU(float*, float*, float*)


*   Host para device: 484.20ms
*   Device para host: 231.46ms
*   Tempo de transferência total: 715.66ms
*   Elementos do vetor: 200000000
*   Taxa de transferêcia: 279463 elementos/ms
*   Tempo de execução do kernel: 735.29ms


  

In [ ]:
%cd /content/src
!nvprof --print-gpu-summary ./sum 512000000

/content/src
./sum Starting...
Vector size 512000000
==13400== NVPROF is profiling process 13400, command: ./sum 512000000
Execution configuration <<<500000, 1024>>>
Arrays match.

==13400== Profiling application: ./sum 512000000
==13400== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.50%  1.25264s         1  1.25264s  1.25264s  1.25264s  [CUDA memcpy DtoH]
                   31.89%  609.83ms         2  304.91ms  302.92ms  306.90ms  [CUDA memcpy HtoD]
                    2.62%  50.019ms         1  50.019ms  50.019ms  50.019ms  sumArraysOnGPU(float*, float*, float*)


*   Host para device: 736.33ms
*   Device para host: 338.87ms
*   Tempo de transferência total: 1.075s
*   Elementos do vetor: 300000000
*   Taxa de transferêcia: 279069 elementos/ms
*   Tempo de execução do kernel: 1.104s


  

## Exemplo 3 // Unroll

_Referência: Capítulo 3 - CUDA Execution Model - Unrolling Loops_

Vamos avaliar agora como o desempenho muda quando fazemos com que uma mesma thread calcule mais de um elemento do vetor. Chamamos essa técnica de _unroll_. Se uma thread é responsável por calcular K elementos do vetor, então dizemos que estamos usando _fator de unroll K_.

Para facilitar os cálculos, vamos usar o fator de unroll sempre na forma de potência de 2. Assim, por exemplo, para um fator de unroll 8, vamos armazenar o valor 3, pois 2^3 = 8.

Se colocarmos o valor do fator de unroll numa variável chamada `unroll`, uma possibilidade de estrutrar o kernel e sua configuração de execução seria da seguinte forma: (perceba que o número total de threads será menor)
```cpp
__global__ void sumArraysOnGPU(float *A, float *B, float *C, int unroll) {
  int i = (blockIdx.x * blockDim.x + threadIdx.x) << unroll;
  int k = i + (1 << unroll);
  for (; i < k; i++)
    C[i] = A[i] + B[i];
}

// Chamando no main...
dim3 block(1024 >> unroll);
dim3 grid(nElem >> 10);
sumArraysOnGPU<<<grid, block>>>(A, B, C, unroll);
```
Dessa forma, suponha que configuramos `unroll` para 3 (2^3 = 8) e `nElem` para 4 K elementos. Teremos cada thread processando 8 elementos do vetor. Uma thread de ID 50 num bloco cujo ID é 2, terá seu valor inicial de i dado por `(2 * 4 + 50) << 3 = 58 << 3 = 464`. Essa thread vai então calcular as posições 464, 465, 466, 467, 468, 469, 470 e 471.

## Tarefa 3 // Unroll

1. Adapte o código da soma de vetores para usar o fator de unroll. Parametrize o número de elementos no vetor, o número de threads por bloco e o fator de unroll, de forma que você possa passar esses valores como argumentos de linha de comando, como segue:
```bash
nvprof ./sum <nElem> <nThreads> <unroll>
```
Use potências de 2 em todos os parâmetros.

In [ ]:
%%writefile /content/src/sumUnroll.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Soma os vetores na CPU
void sumArraysOnHost(float *A, float *B, float *C, const int N) {
  for (int i = 0; i < N; i++)
    C[i] = A[i] + B[i];
}

__global__ void sumArraysOnGPU(float *A, float *B, float *C, int unroll) {
  int i = (blockIdx.x * blockDim.x + threadIdx.x) << unroll;
  int k = i + (1 << unroll);
  for (; i < k; i++)
    C[i] = A[i] + B[i];
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);
  int unroll = atoi(argv[3]);
  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(nThread >> unroll);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C, unroll);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Soma os vetores na CPU para conferir os resultados
  sumArraysOnHost(h_A, h_B, hostRef, nElem);
  // Compara os resultados
  checkResult(hostRef, gpuRef, nElem);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumUnroll.cu


2. Execute seu código com o `nvprof` para cada combinação de quantidade de elementos (32 M, 64 M, 128 M, 256 M) e fator de unroll (1, 2, 4, 8), deixando a quantidade de threads por bloco fixa em algum valor à sua escolha. Elabore um gráfico que mostre a quantidade de somas por segundo em função do fator de unroll. Nesse gráfico, crie uma curva para cada configuração de tamanho do vetor.

Gráfico: https://drive.google.com/open?id=1uYZ1Giqjx976i6ovP8J5hE60sT9XZyL6

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumUnroll sumUnroll.cu

!nvprof --print-gpu-summary ./sumUnroll 32000000 1024 1
!nvprof --print-gpu-summary ./sumUnroll 32000000 1024 2
!nvprof --print-gpu-summary ./sumUnroll 32000000 1024 4
!nvprof --print-gpu-summary ./sumUnroll 32000000 1024 8
!nvprof --print-gpu-summary ./sumUnroll 64000000 1024 1
!nvprof --print-gpu-summary ./sumUnroll 64000000 1024 2
!nvprof --print-gpu-summary ./sumUnroll 64000000 1024 4
!nvprof --print-gpu-summary ./sumUnroll 64000000 1024 8
!nvprof --print-gpu-summary ./sumUnroll 128000000 1024 1
!nvprof --print-gpu-summary ./sumUnroll 128000000 1024 2
!nvprof --print-gpu-summary ./sumUnroll 128000000 1024 4
!nvprof --print-gpu-summary ./sumUnroll 128000000 1024 8
!nvprof --print-gpu-summary ./sumUnroll 256000000 1024 1
!nvprof --print-gpu-summary ./sumUnroll 256000000 1024 2
!nvprof --print-gpu-summary ./sumUnroll 256000000 1024 4
!nvprof --print-gpu-summary ./sumUnroll 256000000 1024 8

/content/src
./sumUnroll Starting...
Vector size 32000000
==11054== NVPROF is profiling process 11054, command: ./sumUnroll 32000000 1024 1
Execution configuration <<<31250, 512>>>
Arrays match.

==11054== Profiling application: ./sumUnroll 32000000 1024 1
==11054== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   67.71%  81.861ms         1  81.861ms  81.861ms  81.861ms  [CUDA memcpy DtoH]
                   29.74%  35.953ms         2  17.976ms  17.789ms  18.164ms  [CUDA memcpy HtoD]
                    2.56%  3.0909ms         1  3.0909ms  3.0909ms  3.0909ms  sumArraysOnGPU(float*, float*, float*, int)
./sumUnroll Starting...
Vector size 32000000
==11068== NVPROF is profiling process 11068, command: ./sumUnroll 32000000 1024 2
Execution configuration <<<31250, 256>>>
Arrays match.

==11068== Profiling application: ./sumUnroll 32000000 1024 2
==11068== Profiling result:
            Type  Time(%)      Time     Calls   

3. Discuta sobre o impacto do fator de unroll no desempenho. Melhorou? Piorou? O que você consegue dizer sobre o motivo dessa mudança no desempenho?



> Como esperado, o desempenho com fator unroll elevado se tornou inferior do que com fator unroll reduzido. Isso acontece porque como cada thread tem que executar mais de uma operação, o processo não ocorre paralelamente em relação a todos os elementos.


## Tarefa 4 // Múltiplas Operações

Vamos expandir a quantidade de cálculos para cada posição do vetor. Podemos estabelecer uma expressão qualquer que terá M operações. Por exemplo, se tivermos:
```cpp
C[i] = A[i] + 3 * B[i] - 2 * A[i] * B[i] + 5 * B[i] * B[i]
```
teremos 8 operações de cálculo para 2 de load e 1 de store. Vamos nos concentrar em alterar a quantidade de operações de cálculo, desconsiderando as de load/store.


1. Modifique o código base do Exemplo 2, deixando parametrizada a quantidade de elementos do vetor, a quantidade de threads por bloco, e a quantidade de operações realizadas, de forma que você possa executar o programa como segue:
```bash
nvprof ./multiops <nElem> <nThreads> <ops>
```

In [ ]:
%%writefile /content/src/sumOps.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Soma os vetores na CPU
void sumArraysOnHost(float *A, float *B, float *C, const int N) {
  for (int i = 0; i < N; i++)
    C[i] = A[i] + B[i];
}

// Soma os vetores na GPU
__global__ void sumArraysOnGPU(float *A, float *B, float *C, int ops) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int a = 0;

  for(int j = 0; j < ops; j++) a++;

  C[i] = A[i] + B[i];
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);
  int ops = atoi(argv[3]);

  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C, ops);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Soma os vetores na CPU para conferir os resultados
  sumArraysOnHost(h_A, h_B, hostRef, nElem);
  // Compara os resultados
  checkResult(hostRef, gpuRef, nElem);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumOps.cu


2. Execute seu código com o `nvprof` para cada combinação de quantidade de elementos (como especificado no exercício 2 da Tarefa 3) e quantidade de operações (5, 10 e 15). Elabore um gráfico que mostre o tempo de execução em função da do tamanho do vetor. Faça uma curva para cada configuração de quantidade de operações.

Gráfico: https://drive.google.com/open?id=1fKbG_PVI7FcRpP5QV64Cq9ZfnhSVSjOP

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumOps sumOps.cu

!nvprof --print-gpu-summary ./sumOps 32000000 1024 5
!nvprof --print-gpu-summary ./sumOps 32000000 1024 10
!nvprof --print-gpu-summary ./sumOps 32000000 1024 15
!nvprof --print-gpu-summary ./sumOps 64000000 1024 5
!nvprof --print-gpu-summary ./sumOps 64000000 1024 10
!nvprof --print-gpu-summary ./sumOps 64000000 1024 15
!nvprof --print-gpu-summary ./sumOps 128000000 1024 5
!nvprof --print-gpu-summary ./sumOps 128000000 1024 10
!nvprof --print-gpu-summary ./sumOps 128000000 1024 15
!nvprof --print-gpu-summary ./sumOps 256000000 1024 5
!nvprof --print-gpu-summary ./sumOps 256000000 1024 10
!nvprof --print-gpu-summary ./sumOps 256000000 1024 15

/content/src
./sumOps Starting...
Vector size 32000000
==11294== NVPROF is profiling process 11294, command: ./sumOps 32000000 1024 5
Execution configuration <<<31250, 1024>>>
Arrays match.

==11294== Profiling application: ./sumOps 32000000 1024 5
==11294== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   67.39%  80.610ms         1  80.610ms  80.610ms  80.610ms  [CUDA memcpy DtoH]
                   29.98%  35.854ms         2  17.927ms  17.622ms  18.232ms  [CUDA memcpy HtoD]
                    2.63%  3.1477ms         1  3.1477ms  3.1477ms  3.1477ms  sumArraysOnGPU(float*, float*, float*, int)
./sumOps Starting...
Vector size 32000000
==11305== NVPROF is profiling process 11305, command: ./sumOps 32000000 1024 10
Execution configuration <<<31250, 1024>>>
Arrays match.

==11305== Profiling application: ./sumOps 32000000 1024 10
==11305== Profiling result:
            Type  Time(%)      Time     Calls       Avg       

3. Discuta os resultados mostrados pelo gráfico quanto ao tempo de execução.

> Como é obvio de se esperar, a medida que a quantidade de operações aumenta, independente da quantidade de elementos, o tempo de execução aumenta proporcionalmente quase que em uma linha reta em todos os casos. Isso ocorre porque por mais que emnquanto um thread esteja disponível para executar diferentes operações simultaneamente, o número de operações continua impactando no tempo de execução final.

## Exemplo 5 // Acesso à Memória Global

_Referência: Capítulo 4 - Global Memory - Memory Access Patterns_

Os acessos à memória global são feitos com a ajuda de caches, e essa memória global é um espaço de memória que você consegue acessar a partir de seus kernels. O padrão ideal de acessos à cache é quando esse acesso é feito de forma _aligned_ (alinhada) e _coalesced_ (agrupada).

Acesso alinhado ocorre quando o primeiro endereço acessado numa transferência é um múltiplo par da granularidade de cache usada nessa transferência - 32 B para L2 ou 128 B para L1. Um acesso desalinhado desperdiça os dados carregados.

Acesso agrupado occore quando todas as 32 threads em um _warp_ (warp é o conjunto de threads sendo executadas simultaneamente) acessam uma área contígua de memória.

![Acesso alinhado e agrupado](https://docs.google.com/uc?export=download&id=116WeX9Ng7J_mgIoCwr5f9mQfTx65F48X)


## Tarefa 5 // Acesso à Memória Global

Vamos explorar o que acontece quando quebramos os acessos alinhados e agrupados. 

Primeiramente, vejamos o que acontece se acessarmos posições mais espaçadas dos vetores do exemplo de soma. Vamos chamar esse espaçamento de pulos, ou _strikes_ no acesso, e seu valor será em potências de 2.

Faremos a soma de uma forma diferente: cada posição `C[i]` terá o resultado da soma das posições `A[i << strike] + B[i << strike]`. Por exemplo, se tivermos o _strike_ igual a 3 (2^3 = 8), teremos o seguinte calculo `C[6] = A[48] + B[48]`. Na próxima posição, teremos `C[7] = A[56] + B[56]`. Note que muitas posições dos vetores A e B foram puladas, e note também que o vetor C será menor que os vetores A e B. Especificamente, seu tamanho será `N >> strike`.

1. Adapte o código de soma de vetores considerando o _strike_. Deixe parametrizado o tamanho dos vetores A e B, o número de threads por bloco, e o valor do _strike_. 

In [ ]:
%%writefile /content/src/sumStrike.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Soma os vetores na GPU
__global__ void sumArraysOnGPU(float *A, float *B, float *C, int strike, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  
  int j = i << strike;

  if(j>=n){
    C[i] = A[i] + B[i];
  } else {
    C[i] = A[j] + B[j];
  }
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);
  int strike = atoi(argv[3]);

  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C, strike, nElem);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumStrike.cu


2. Execute seu programa com o `nvprof` usando diferentes combinações de quantidade de elementos para os vetores A e B (os mesmos valores do exercício 2 da Tarefa 3) e diferentes valores de _strike_ (4, 8, 16, 32, 64 e 128). Elabore um gráfico que mostre o tempo de execução em função do tamanho dos vetores, fazendo uma curva para cada valor de _strike_. Use todos os parâmetros como potências de 2.

Gráfico: https://drive.google.com/open?id=12wfYKBUK7RiaTc_EWNfi90kfQr9hB3LP

In [ ]:
# Reduzi o tamanho do vetor em 10000 vezes, pois iria demorar tempo demais para
# executar

%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumStrike sumStrike.cu

!nvprof --print-gpu-summary ./sumStrike 3200 1024 4
!nvprof --print-gpu-summary ./sumStrike 3200 1024 8
!nvprof --print-gpu-summary ./sumStrike 3200 1024 16
!nvprof --print-gpu-summary ./sumStrike 3200 1024 32
!nvprof --print-gpu-summary ./sumStrike 3200 1024 64
!nvprof --print-gpu-summary ./sumStrike 3200 1024 128
!nvprof --print-gpu-summary ./sumStrike 6400 1024 4
!nvprof --print-gpu-summary ./sumStrike 6400 1024 8
!nvprof --print-gpu-summary ./sumStrike 6400 1024 16
!nvprof --print-gpu-summary ./sumStrike 6400 1024 32
!nvprof --print-gpu-summary ./sumStrike 6400 1024 64
!nvprof --print-gpu-summary ./sumStrike 6400 1024 128
!nvprof --print-gpu-summary ./sumStrike 12800 1024 4
!nvprof --print-gpu-summary ./sumStrike 12800 1024 8
!nvprof --print-gpu-summary ./sumStrike 12800 1024 16
!nvprof --print-gpu-summary ./sumStrike 12800 1024 32
!nvprof --print-gpu-summary ./sumStrike 12800 1024 64
!nvprof --print-gpu-summary ./sumStrike 12800 1024 128
!nvprof --print-gpu-summary ./sumStrike 25600 1024 4
!nvprof --print-gpu-summary ./sumStrike 25600 1024 8
!nvprof --print-gpu-summary ./sumStrike 25600 1024 16
!nvprof --print-gpu-summary ./sumStrike 25600 1024 32
!nvprof --print-gpu-summary ./sumStrike 25600 1024 64
!nvprof --print-gpu-summary ./sumStrike 25600 1024 128

/content/src
./sumStrike Starting...
Vector size 3200
==13655== NVPROF is profiling process 13655, command: ./sumStrike 3200 1024 4
Execution configuration <<<3, 1024>>>
==13655== Profiling application: ./sumStrike 3200 1024 4
==13655== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.44%  8.9280us         2  4.4640us  4.3840us  4.5440us  [CUDA memcpy HtoD]
                   26.39%  4.8640us         1  4.8640us  4.8640us  4.8640us  sumArraysOnGPU(float*, float*, float*, int, int)
                   25.17%  4.6400us         1  4.6400us  4.6400us  4.6400us  [CUDA memcpy DtoH]
./sumStrike Starting...
Vector size 3200
==13666== NVPROF is profiling process 13666, command: ./sumStrike 3200 1024 8
Execution configuration <<<3, 1024>>>
==13666== Profiling application: ./sumStrike 3200 1024 8
==13666== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50

3. Discuta o impacto do _strike_ no desempenho.

> Como é possível notar no gráfico já apresentado, o tempo de execução aumenta bastante com o aumento do strike. Embora que não esteja sendo apresentado no gráfico, o tempo de execução das somas continua sendo o mesmo (ou quase o mesmo) do mesmo algoritmo sem strike, no entanto, como as posições dos vetores não são exploradas de forma contígua, o princípio da localidade acaba sendo cada vez menos explorado a medida que o número do strike aumenta. Portanto, o fato de o novo tempo de execução ser superior é provocado pela taxa de transferência de dados entre CPU e GPU.

Para os próximos exercícios desta tarefa, vamos simular um acesso aleatório nas posições do vetor, usando novamente a soma de vetores como base, com is vetores A, B e C de mesmo tamanho. Para isso, vamos usar um vetor a mais, com o mesmo tamanho dos outros, chamado R. Este vai mapear uma posição em outra, aleatória. Você deve montar esse vetor de forma que o conteúdo de cada um de seus elementos seja um índice, entre 0 e N - 1, porém esses valores devem ser "sorteados" no vetor.

Fazendo isso, sempre que você acessar alguma posição dos vetores A, B ou C, ao invés de usar diretamente o índice `i`, você vai usar o índice `R[i]`:
```cpp
// No corpo do for da função sumArraysOnHost
//  ou no corpo do kernel sumArraysOnGPU
C[R[i]] = A[R[i]] + B[R[i]];
```
4. Altere o código para que seja criado esse vetor `R`, e deixe parametrizado: o número de elementos nos vetores, o número de threads por bloco e uma opção para se usar ou não o vetor `R`.

In [ ]:
%%writefile /content/src/sumR.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para conferir se as somas na CPU e na GPU correspondem
void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  for (int i=0; i<N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      printf("Arrays do not match!\n");
      printf("host %5.2f gpu %5.2f at current %d\n", hostRef[i], gpuRef[i], i);
      return;
    }
  }
  printf("Arrays match.\n\n");
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialDataInt(int *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (int)(rand() % n);
}

// Soma os vetores na CPU
void sumArraysOnHost(float *A, float *B, float *C, const int N, int * R, int use) {
  for (int i = 0; i < N; i++)
    {
        if(use==0) {
          C[i] = A[i] + B[i];
        } else {
          C[i] = A[R[i]] + B[R[i]];
        }
    }
}

// Soma os vetores na GPU
__global__ void sumArraysOnGPU(float *A, float *B, float *C, int *R, int use) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;

  if(use==0) {
    C[i] = A[i] + B[i];
  } else {
    C[i] = A[R[i]] + B[R[i]];
  }
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);
  int use = atoi(argv[3]);

  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  int *h_R;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  h_R = (int *)malloc(nElem * sizeof(int));

  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  int *d_R;

  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);
  cudaMalloc((int **)&d_R, nElem * sizeof(int));

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  cudaMemcpy(d_R, h_R, nElem * sizeof(int), cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  sumArraysOnGPU<<<grid, block>>>(d_A, d_B, d_C, d_R, use);
  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Soma os vetores na CPU para conferir os resultados
  sumArraysOnHost(h_A, h_B, hostRef, nElem, h_R, use);
  // Compara os resultados
  checkResult(hostRef, gpuRef, nElem);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumR.cu


5. Execute com o `nvprof` com cada combinação de quantidade de elementos (mesmos valores do exercício 2 da Tarefa 3) e opção de usar ou não o vetor `R`. Elabore um gráfico do tempo de execução em função do tamanho do vetor, fazendo uma curva para o acesso direto dos índices e outra para o acesso via o vetor aleatório `R`.

Gráfico: https://drive.google.com/open?id=1XXiba0nASpDntFe_iAabOSWmlOimwX2h

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumR sumR.cu

!nvprof --print-gpu-summary ./sumR 32000000 1024 0
!nvprof --print-gpu-summary ./sumR 32000000 1024 1
!nvprof --print-gpu-summary ./sumR 64000000 1024 0
!nvprof --print-gpu-summary ./sumR 64000000 1024 1
!nvprof --print-gpu-summary ./sumR 128000000 1024 0
!nvprof --print-gpu-summary ./sumR 128000000 1024 1
!nvprof --print-gpu-summary ./sumR 256000000 1024 0
!nvprof --print-gpu-summary ./sumR 256000000 1024 1

/content/src
./sumR Starting...
Vector size 32000000
==11521== NVPROF is profiling process 11521, command: ./sumR 32000000 1024 0
Execution configuration <<<31250, 1024>>>
Arrays match.

==11521== Profiling application: ./sumR 32000000 1024 0
==11521== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   55.87%  86.927ms         1  86.927ms  86.927ms  86.927ms  [CUDA memcpy DtoH]
                   42.09%  65.486ms         3  21.829ms  18.234ms  29.006ms  [CUDA memcpy HtoD]
                    2.05%  3.1877ms         1  3.1877ms  3.1877ms  3.1877ms  sumArraysOnGPU(float*, float*, float*, int*, int)
./sumR Starting...
Vector size 32000000
==11532== NVPROF is profiling process 11532, command: ./sumR 32000000 1024 1
Execution configuration <<<31250, 1024>>>
Arrays match.

==11532== Profiling application: ./sumR 32000000 1024 1
==11532== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min     

6. Discuta sobre o impacto do acesso aleatório no desempenho.

> É levemente perceptível no gráfico já apresentado que a linha indicadora do tempo de execução do algoritmo sem vetor de números aleatórios está acima da linha indicadora do algoritmo com vetor de números aleatórios. Isso ocorre porque quando não se calcula previamente os valores aleatórios, a GPU deixa de executar totalmente em paralelo, já que tem que despender parte de seus recursos para calcular o número aleatório almejado. No entanto, comparado qualquer um dos dois algoritmos com um que não faça uso de acesso aleatório de qualquer espécie, ambos os dois algoritmos demorarão muito mais para serem executados, já que eles fazem muito mal aproveitamento do princípio de localidade da cache.

## Exemplo 6 // Memória Compartilhada

_Referência: Capítulo 5 - Shared Memory and Constant Memory_

A memória compartilhada - _shared memory_ - pode ser vista como uma memória cache gerenciada pelo programador, e geralmente serve também como canal de comunicação intrabloco para as threads, ou mesmo uma memória _scratchpad_ para auxiliar na transformação de dados.

Para atingir maiores taxas de transferência, essa memória é organizada em 32 módulos de memória de mesmo tamanho que podem ser acessados simultaneamente, chamados bancos. São 32 porque há 32 threads em um _warp_. Se uma operação de load ou store lançada num _warp_ não acessa mais de um endereço de memória por banco, a operação é servida com apenas uma transferência de memória. Do contrário, as transferências são serializadas, diminuindo o aproveitamento da taxa de transferência.

![Acessos na memória compartilhada](https://docs.google.com/uc?export=download&id=1aSip-o4c5J9WorgJV3Lar9ftnqlpRfia)

Na figura, vemos primeiro um caso onde há apenas um acesso num mesmo banco, e depois um caso de muitos acessos em um mesmo banco.

## Tarefa 6 // Memória Compartilhada

Nesta tarefa, estamos interessados apenas no impacto do desempenho causado pelo conflito de bancos, não na operação em si que será realizada na GPU. Portanto, você pode usar a código de soma de vetores como base, mas não usaremos a função que verifica os resultados, nem precisamos repetir os cálculos na CPU, já que não faremos comparações.

Vejamos como usar a memória compartilhada. Dentro de um kernel, se cada thread acessar um banco diferente, como segue, não haverá conflito de bancos:
```cpp
__shared__ float smem[512];
smem[threadIdx.x] += 1.0;
```
Mas se fizermos com que múltiplas threads acessem um mesmo banco, em posições diferentes, teremos conflito:
```cpp
__shared__ float smem[512];
smem[threadIdx.x * 2] += 1.0;
```
Nesse caso, se lançarmos o kernel com 32 threads, a thread 0 e a thread 16 serão mapeadas para as posições 0 e 32, respectivamente, que estão no mesmo banco de memória compartilhada, o banco 0.

1. Elabore um código em que você possa testar o conflito nos bancos de forma parametrizada (pode ser com base na soma de vetores ou não). Você pode deixar fixa a quantidade de threads em 32.

In [ ]:
%%writefile /content/src/sumConflit.cu
#include <cuda_runtime.h>
#include <stdio.h>

 __shared__ float smem[32];

// Soma os vetores na GPU
__global__ void confliGPU(int conflit) {
  float a = 3.878 + 2.76;
  smem[(threadIdx.x * conflit) % 32] += a;

}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  int nThread = atoi(argv[1]);
  int conflit = atoi(argv[2]);

  // Executa a soma no device
  confliGPU<<<1, 32>>>(conflit);

  return 0;
}

Overwriting /content/src/sumConflit.cu


2. Execute seu código com o `nvprof` para alguns valores de quantidade de conflitos (2, 4, 8 e 16). Elabore um gráfico que mostre o tempo de execução em função da quantidade de conflitos. 

Gráfico: https://drive.google.com/open?id=1TCeosStLEdLJ_M9GuCZDxKdW3QF3nNMy

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumConflit sumConflit.cu

!nvprof --print-gpu-summary ./sumConflit 32 1
!nvprof --print-gpu-summary ./sumConflit 32 2
!nvprof --print-gpu-summary ./sumConflit 32 4
!nvprof --print-gpu-summary ./sumConflit 32 8
!nvprof --print-gpu-summary ./sumConflit 32 16

/content/src
./sumConflit Starting...
==16549== NVPROF is profiling process 16549, command: ./sumConflit 32 1
==16549== Profiling application: ./sumConflit 32 1
==16549== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.8160us         1  2.8160us  2.8160us  2.8160us  confliGPU(int)
./sumConflit Starting...
==16563== NVPROF is profiling process 16563, command: ./sumConflit 32 2
==16563== Profiling application: ./sumConflit 32 2
==16563== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.8160us         1  2.8160us  2.8160us  2.8160us  confliGPU(int)
./sumConflit Starting...
==16574== NVPROF is profiling process 16574, command: ./sumConflit 32 4
==16574== Profiling application: ./sumConflit 32 4
==16574== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.

3. Discuta sobre o impacto no desempenho com o aumento dos conflitos.

> Com os conflitos, o tempo de execução tendem a não se reduzir ou até mesmo 
aumetarem. Isso acontece porque quando um conflito acontece, o hardware é reposnsável de trata-los evitando que algum erro de baixo nível aconteça. No entanto, essse tratamento dispende tempo, ou seja, quanto mais conflitos, mais tempo é gasto para resolvê-los.

## Tarefa 7 // Registros

As variáveis locais e as variáveis de "controle" - `threadIdx`, `blockIdx`, etc. - do kernel são preferencialmente alocadas em registradores. Caso se tenha uma demanda maior que a quantidade de registradores alocados para uma thread, essas variáveis podem ser alocadas na memória global de forma transparente para o programador, porém isso pode gerar perda de desempenho, mesmo com a ajuda das caches L1 e L2.

Vamos ver esse impacto criando mais variáveis locais no kernel. Suponha que temos um vetor local, como segue:
```cpp
__global__ void sumArraysOnGPU(float *A, float *B, float *C, int N) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  float local[4];
  for (int j = 0; j < 4; j++)
    local[j] = 2 * A[(i + j) % N];
  C[i] = A[i] + B[i] + local[i % 4];
}
```
Note que, assim como na tarefa anterior, não estamos mais interessados na operação em si realizada, vamos apenas ver as alterações no desempenho.

1. Modifique o código da soma de vetores para usar o vetor local como mostrado. Faça também uma outra versão do kernel, no mesmo código, com outro nome, e que use quatro variáveis locais ao invés de um vetor de quatro posições. Deixe seu código parametrizado de forma que se possa escolher qual desses dois kernels será chamado.

In [ ]:
%%writefile /content/src/sumVariableVector.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

__global__ void sumArraysOnGPUVector(float *A, float *B, float *C, int N) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  float local[4];
  for (int j = 0; j < 4; j++)
    local[j] = 2 * A[(i + j) % N];
  C[i] = A[i] + B[i] + local[i % 4];
}

__global__ void sumArraysOnGPUVariables(float *A, float *B, float *C, int N) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  float local0, local1, local2, local3;

  local0 = 2 * A[(i + 0) % N];
  local1 = 2 * A[(i + 1) % N];
  local2 = 2 * A[(i + 2) % N];
  local3 = 2 * A[(i + 3) % N];

  C[i] = A[i] + B[i] + local0 + local1 + local2+ local3;
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);
  int use = atoi(argv[3]);

  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);
  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  if(use==1) {
      sumArraysOnGPUVector<<<grid, block>>>(d_A, d_B, d_C, nElem);
      printf("Vector allocated in GPU\n");
  } else {
      printf("Variables allocated in GPU\n");
      sumArraysOnGPUVariables<<<grid, block>>>(d_A, d_B, d_C, nElem);
  }

  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumVariableVector.cu


2. Escolha um tamanho de vetor, de grade, e de bloco, e execute seu código duas vezes, um para cada kernel. Compare os tempos de execução e discuta sobre.

> É possível notar que quando utilizadas 4 varáveis ao invés de um vetor de 4 posições o desempenho foi bem pior. Isso foi causado tanto pelo demorado acesso a memoria global quanto pela não exploração do princípio de localidade.

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumVariableVector sumVariableVector.cu

!nvprof --print-gpu-summary ./sumVariableVector 1024 32 1
!nvprof --print-gpu-summary ./sumVariableVector 1024 32 0

/content/src
./sumVariableVector Starting...
Vector size 1024
==323== NVPROF is profiling process 323, command: ./sumVariableVector 1024 32 1
Vector allocated in GPU
Execution configuration <<<1, 1024>>>
==323== Profiling application: ./sumVariableVector 1024 32 1
==323== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   43.02%  6.1120us         1  6.1120us  6.1120us  6.1120us  sumArraysOnGPUVector(float*, float*, float*, int)
                   36.49%  5.1840us         2  2.5920us  2.4320us  2.7520us  [CUDA memcpy HtoD]
                   20.50%  2.9120us         1  2.9120us  2.9120us  2.9120us  [CUDA memcpy DtoH]
./sumVariableVector Starting...
Vector size 1024
==334== NVPROF is profiling process 334, command: ./sumVariableVector 1024 32 0
Variables allocated in GPU
Execution configuration <<<1, 1024>>>
==334== Profiling application: ./sumVariableVector 1024 32 0
==334== Profiling result:
            Type  Time(%)  

3. Modifique novamente o código, dessa vez alocando um vetor de quatro posições no `main` e passando-o como argumento para o kernel.

In [ ]:
%%writefile /content/src/sumVectorMain.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para atribuir valores iniciais aos vetores de floats
void initialData(float *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (float)(rand() & 0xFF) / 10.0;
}

__global__ void sumArraysOnGPUVector(float *A, float *B, float *C, float *local, int N) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  
  for (int j = 0; j < 4; j++)
    local[j] = 2 * A[(i + j) % N];
  C[i] = A[i] + B[i] + local[i % 4];
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);

  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(float);
  float *h_A, *h_B, *h_local, *hostRef, *gpuRef;
  h_A = (float *)malloc(nBytes);
  h_B = (float *)malloc(nBytes);

  // Vector for question
  h_local = (float *) malloc(4 * sizeof(float));

  hostRef = (float *)malloc(nBytes);
  gpuRef = (float *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  float *d_A, *d_B, *d_C, *d_local;
  cudaMalloc((float **)&d_A, nBytes);
  cudaMalloc((float **)&d_B, nBytes);
  cudaMalloc((float **)&d_C, nBytes);

  // Vector for question
  cudaMalloc((float **)&d_local, 4 * sizeof(float));

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Vector for question
  cudaMemcpy(d_local, h_local, 4 * sizeof(float), cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  // Executa a soma no device
  printf("Vector allocated in Main Code\n");
  sumArraysOnGPUVector<<<grid, block>>>(d_A, d_B, d_C, d_local, nElem);

  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);
  free(hostRef);
  free(gpuRef);

  return 0;
}

Overwriting /content/src/sumVectorMain.cu


4. Execute com a mesma configuração de tamanhos no exercício 2 e compare o tempo de execução com os tempos previamente obtidos.

> Embora que tenha sido necessário enviar para o Kernel uma quantidade maior de dados, o tempo de execução foi tão bom quanto do melhor caso explorado no exercício 2. Isso provavelmente ocorreu porque não foi necessário um acesso a memória global e o princípio de localidade foi explorado.

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o sumVectorMain sumVectorMain.cu

!nvprof --print-gpu-summary ./sumVectorMain 1024 32

/content/src
./sumVectorMain Starting...
Vector size 1024
==520== NVPROF is profiling process 520, command: ./sumVectorMain 1024 32
Vector allocated in Main Code
Execution configuration <<<1, 1024>>>
==520== Profiling application: ./sumVectorMain 1024 32
==520== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   42.08%  6.7200us         3  2.2400us  1.5360us  2.7520us  [CUDA memcpy HtoD]
                   40.08%  6.4000us         1  6.4000us  6.4000us  6.4000us  sumArraysOnGPUVector(float*, float*, float*, float*, int)
                   17.84%  2.8480us         1  2.8480us  2.8480us  2.8480us  [CUDA memcpy DtoH]


## Tarefa 8 // Shuffle

A GPU oferece também a instrução _shuffle_ para troca de valores entre threads num mesmo _warp_. Vamos comparar o desempenho do _shuffle_ com os outros recursos de memória como _shared memory_.

O kernel a seguir usa a função `__shfl` para passar o valor da posição `A[i]` para a posição `B[i+1]` (perceba a soma `+1` no segundo argumento), fazendo um `loop` a cada 32 elementos (terceiro argumento).
```cpp
__global__ void shuffle(int *A, int *B) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int val = A[i];
  val = __shfl(val, threadIdx.x + 1, 32);
  B[i] = val;
}
```
Assim, supondo que os vetores A e B sejam de tamanho 64, esses seriam seus valores após a execução do kernel:
```
A = 0, 1, 2, ..., 30, 31, 32, 33, ..., 62, 63
B = 1, 2, 3, ..., 31,  0, 33, 34, ..., 63, 32
```

1. Elabore o código completo que usa o kernel mostrado.

In [ ]:
%%writefile /content/src/shuffleWarp.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Use esta função para atribuir valores iniciais aos vetores de ints
void initialData(int *a, int n) {
  srand(time(NULL));
  for (int i = 0; i < n; i++)
    a[i] = (int)(rand()%100);
}

__global__ void shuffle(int *A, int *B) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int val = A[i];
  val = __shfl_sync(val, threadIdx.x + 1, 32);
  B[i] = val;
}

int main(int argc, char **argv) {
  printf("%s Starting...\n", argv[0]);
  
  // Configura o tamanho dos vetores
  int nElem = atoi(argv[1]);
  int nThread = atoi(argv[2]);

  printf("Running shuffle shared memory\n");
  printf("Vector size %d\n", nElem);
  
  // Aloca memória no host
  size_t nBytes = nElem * sizeof(int);
  int *h_A, *h_B;
  h_A = (int *)malloc(nBytes);
  h_B = (int *)malloc(nBytes);

  // Inicializa os dados ainda no host
  initialData(h_A, nElem);
  initialData(h_B, nElem);
  
  // Aloca memória global na GPU (device)
  int *d_A, *d_B;
  cudaMalloc((int **)&d_A, nBytes);
  cudaMalloc((int **)&d_B, nBytes);

  // Transfere os dados do host (CPU) para o device (GPU)
  cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice);

  // Configura os tamanho de:
  //   bloco (quantas threads por bloco?)
  //   e grade (quantos blocos por grade?)
  dim3 block(1024);
  dim3 grid(nElem >> 10);

  shuffle<<<grid, block>>>(d_A, d_B);

  printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);

  // Copia o resultado do kernel do device para o host
  //cudaMemcpy(h_A, d_A, nBytes, cudaMemcpyDeviceToHost);
  //cudaMemcpy(h_B, d_B, nBytes, cudaMemcpyDeviceToHost);

  // Libera a memória da GPU
  cudaFree(d_A);
  cudaFree(d_B);

  // Libera a memória da CPU
  free(h_A);
  free(h_B);

  return 0;
}

Overwriting /content/src/shuffleWarp.cu


2. Elabore o código que implementa a mesma funcionalidade, porém usando memória compartilhada.

In [ ]:
%%writefile /content/src/shuffleSharedMemory.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <cuda_profiler_api.h>
#include <stdio.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>


__inline__ __device__
int warpReduceSum(int val) {
    for (int offset = 16; offset > 0; offset /= 2)
        val += __shfl_down(val, offset);
    return val;
}

__inline__ __device__
int blockReduceSum(int val) {
    static __shared__ int shared[32];
    int lane = threadIdx.x%32;
    int wid = threadIdx.x / 32;
    val = warpReduceSum(val);

    //write reduced value to shared memory
    if (lane == 0) shared[wid] = val;
    __syncthreads();

    //ensure we only grab a value from shared memory if that warp existed
    val = (threadIdx.x<blockDim.x / 32) ? shared[lane] : int(0);
    if (wid == 0) val = warpReduceSum(val);

    return val;
}

__global__ void device_reduce_stable_kernel(int *in, int* out, int N) {
    int sum = int(0);
    //printf("value = %d ", blockDim.x*gridDim.x);
    for (int i = blockIdx.x*blockDim.x + threadIdx.x; i<N; i += blockDim.x*gridDim.x) {
        sum += in[i];
    }
    sum = blockReduceSum(sum);
    if (threadIdx.x == 0)
        out[blockIdx.x] = sum;
}

void device_reduce_stable(int *in, int* out, int N) {
    //int threads = 512;
    //int blocks = min((N + threads - 1) / threads, 1024);
    const int maxThreadsPerBlock = 1024;
    int threads = maxThreadsPerBlock;
    int blocks = N / maxThreadsPerBlock;
    device_reduce_stable_kernel << <blocks, threads >> >(in, out, N);
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
        printf("Error: %s\n", cudaGetErrorString(err));

    device_reduce_stable_kernel << <1, 1024 >> >(out, out, blocks);
    //cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
        printf("Error: %s\n", cudaGetErrorString(err));
}

__global__ void global_reduce_kernel(int * d_out, int * d_in)
{
    int myId = threadIdx.x + blockDim.x * blockIdx.x;
    int tid = threadIdx.x;

    // do reduction in global mem
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
        {
            d_in[myId] += d_in[myId + s];
        }
        __syncthreads();        // make sure all adds at one stage are done!
    }

    // only thread 0 writes result for this block back to global mem
    if (tid == 0)
    {
        d_out[blockIdx.x] = d_in[myId];
    }
}

__global__ void shmem_reduce_kernel(int * d_out, const int * d_in)
{
    // sdata is allocated in the kernel call: 3rd arg to <<<b, t, shmem>>>
    extern __shared__ int sdata[];

    int myId = threadIdx.x + blockDim.x * blockIdx.x;
    int tid = threadIdx.x;

    // load shared mem from global mem
    sdata[tid] = d_in[myId];
    __syncthreads();            // make sure entire block is loaded!

    // do reduction in shared mem
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
        {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();        // make sure all adds at one stage are done!
    }

    // only thread 0 writes result for this block back to global mem
    if (tid == 0)
    {
        d_out[blockIdx.x] = sdata[0];
    }
}

void reduce(int * d_out, int * d_intermediate, int * d_in,
    int size, bool usesSharedMemory)
{
    // assumes that size is not greater than maxThreadsPerBlock^2
    // and that size is a multiple of maxThreadsPerBlock
    const int maxThreadsPerBlock = 1024;
    int threads = maxThreadsPerBlock;
    int blocks = size / maxThreadsPerBlock;
    if (usesSharedMemory)
    {
        shmem_reduce_kernel << <blocks, threads, threads * sizeof(int) >> >
            (d_intermediate, d_in);
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess)
            printf("Error: %s\n", cudaGetErrorString(err));
    }
    else
    {
        global_reduce_kernel << <blocks, threads >> >
            (d_intermediate, d_in);
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess)
            printf("Error: %s\n", cudaGetErrorString(err));
    }
    // now we're down to one block left, so reduce it
    threads = blocks; // launch one thread for each block in prev step
    blocks = 1;
    if (usesSharedMemory)
    {
        shmem_reduce_kernel << <blocks, threads, threads * sizeof(int) >> >
            (d_out, d_intermediate);
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess)
            printf("Error: %s\n", cudaGetErrorString(err));
    }
    else
    {
        global_reduce_kernel << <blocks, threads >> >
            (d_out, d_intermediate);
        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess)
            printf("Error: %s\n", cudaGetErrorString(err));
    }
}

int main(int argc, char **argv)
{
    const int ARRAY_SIZE = atoi(argv[1]);
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
    printf("%s Starting...\n", argv[0]);

    // generate the input array on the host
    int h_in[ARRAY_SIZE];
    int sum = 0.0f;
    for (int i = 0; i < ARRAY_SIZE; i++) {
        // generate random int in [-1.0f, 1.0f]
        h_in[i] = i;
        sum += h_in[i];
    }

    // declare GPU memory pointers
    int * d_in, *d_intermediate, *d_out;

    // allocate GPU memory
    cudaMalloc((void **)&d_in, ARRAY_BYTES);
    cudaMalloc((void **)&d_intermediate, ARRAY_BYTES); // overallocated
    cudaMalloc((void **)&d_out, sizeof(int));

    // transfer the input array to the GPU
    cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    // launch the kernel
    cudaProfilerStart();
    
    printf("Running shuffle shared memory\n");
    printf("Vector size %d\n", ARRAY_SIZE);
    cudaEventRecord(start, 0);
    
    reduce(d_out, d_intermediate, d_in, ARRAY_SIZE, true);
    
    cudaEventRecord(stop, 0);
       
    cudaProfilerStop();
    cudaEventSynchronize(stop);
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    elapsedTime /= 100.0f;      // 100 trials

    // copy back the sum from GPU
    //int h_out;
    //cudaMemcpy(&h_out, d_out, sizeof(int), cudaMemcpyDeviceToHost);

    //printf("average time elapsed: %f\n", elapsedTime);

    // free GPU memory allocation
    cudaFree(d_in);
    cudaFree(d_intermediate);
    cudaFree(d_out);

    return 0;

}

Overwriting /content/src/shuffleSharedMemory.cu


3. Execute os dois programas com o `nvprof` com diferentes configurações de conflito para o código da memória compartilhada: sem conflitos, 2, 4 e 8 conflitos. Fixe a quantidade de threads por bloco em 32 e a quantidade de elementos no vetor a um valor à sua escolha (no mínimo 4096). Discuta sobre o desempenho de cada execução.

In [ ]:
%cd /content/src
!nvcc -O2 -arch=sm_35 -o shuffleWarp shuffleWarp.cu
!nvcc -O2 -arch=sm_35 -o shuffleSharedMemory shuffleSharedMemory.cu

!echo "----------------------------------------------------------------------------------------------------------------"

!nvprof --print-gpu-summary ./shuffleWarp 4096 1024
!nvprof --print-gpu-summary ./shuffleSharedMemory 4096

# Referencia: https://stackoverflow.com/questions/44278317/cuda-shuffle-instruction-reduction-slower-than-shared-memory-reduction

/content/src
shuffleSharedMemory.cu(13): warning: function "__shfl_down(int, unsigned int, int)"
/usr/local/cuda/bin/../targets/x86_64-linux/include/sm_30_intrinsics.hpp(195): here was declared deprecated ("__shfl_down() is deprecated in favor of __shfl_down_sync() and may be removed in a future release (Use -Wno-deprecated-declarations to suppress this warning).")

ptxas /tmp/tmpxft_00000a02_00000000-5_shuffleSharedMemory.ptx, line 58; warning : Instruction 'shfl' without '.sync' is deprecated since PTX ISA version 6.0 and will be discontinued in a future PTX ISA version
ptxas /tmp/tmpxft_00000a02_00000000-5_shuffleSharedMemory.ptx, line 63; warning : Instruction 'shfl' without '.sync' is deprecated since PTX ISA version 6.0 and will be discontinued in a future PTX ISA version
ptxas /tmp/tmpxft_00000a02_00000000-5_shuffleSharedMemory.ptx, line 68; warning : Instruction 'shfl' without '.sync' is deprecated since PTX ISA version 6.0 and will be discontinued in a future PTX ISA version
p

> É notável que o Warp apresenta desempenho bem superior ao SharedMemory, pórém isso não é uma surpresa, pois, enquanto o SharedMemory é uma implementação manual da solução do problema acima citado e, por isso, acaba consumindo mais recursos por estar níveis acima do nível que o warp está impleementado, pois é um recurso disponível nas GPU's. Sendo assim, alguns tratamentos que já são automaticamente tratados utilizando warp, terão de ser tratados de forma convencional, o que torna a execução do SharedMemory bem mais lenta.